# Discover & Process Sentinel-1 InSAR pairs

- The provider used for this notebook is `cop_dataspace`
- A free account needs to be created at https://dataspace.copernicus.eu/
- The credentials need to be configured as explained here https://eodag.readthedocs.io/en/stable/getting_started_guide/configure.html

In [ ]:
%load_ext autoreload
%autoreload 2

from eo_tools.util import explore_products
import geopandas as gpd
import folium
import folium.plugins
import logging
import os

from eodag import EODataAccessGateway

# credentials need to be stored in the following file (see EODAG docs)
confpath = "/data/eodag_config.yml"
dag = EODataAccessGateway(user_conf_file_path=confpath)
# make sure cop_dataspace will be used 
dag.set_preferred_provider("cop_dataspace")
logging.basicConfig(level=logging.INFO)

## Select an area of interest

- Select a drawing tool (polygon, circle or rectangle)
- Draw the AOI on the map
- Click "Export" on the right to save the GeoJSON file

In [ ]:
map_select = folium.Map()
folium.plugins.Draw(
    export=True,
    draw_options={
        "polyline": False,
        "circlemarker": False,
        "marker": False,
    },
).add_to(map_select)
map_select

## Search

In [ ]:
# load a geometry
file_aoi = "https://raw.githubusercontent.com/odhondt/eo_tools/refs/heads/main/data/Morocco_AOI.geojson"
shp = gpd.read_file(file_aoi).geometry[0]
search_criteria = {
    "productType": "S1_SAR_SLC",
    "start": "2023-09-01",
    "end": "2023-09-20",
    "geom": shp
}
results, _ = dag.search(**search_criteria)

## Visualize by groups

- For InSAR, products must be overlapping and from the same relative orbit.
- They also have to be of the same ascending / descending configuration.
- The following function can be used to visually find such products 

In [ ]:
explore_products(results, shp)

## Download

In [ ]:
# setup a data directory
data_dir = "/data/S1"

# fill indices of products to download
indices_to_download = [1, 0]
to_download = [results[i] for i in indices_to_download]

# if the data has already been downloaded, just re-run the cell to get the product paths
dl = dag.download_all(to_download, output_dir=data_dir, extract=False)